In [1]:
from __future__ import print_function, division
import numpy as np
import astropy.units as u
from pint.models.timing_model import Component
import pint.models.parameter
from astropy import log
import ephem
import pint.observatory
import pint.toa
import pint.models
import pint.fitter
import pint.utils
from pint.utils import dmxrange
import pint.models.dispersion_model
import pprint
from fractions import Fraction
import skyfield
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord, get_sun
from astropy.visualization import quantity_support
from pint.models.timing_model import DelayComponent
import pint.models.parameter

from __future__ import division

quantity_support()
from array import array

In [2]:
#Get your TOAs from the tim file (in this case from pulsar B1855)
#Put these TOAs into a PINT object so we can work with them
ts1855 = pint.toa.get_TOAs("B1855+09_NANOGrav_dfg+12.tim", usepickle=True)

#Take the pulsar timing model from the par file and put it into a PINT object
m1855 = pint.models.get_model("B1855+09_NANOGrav_dfg+12_TAI.par")

#Remove a component called "DispersionDMX" from the 
m1855.remove_component("DispersionDMX")

glsfit = pint.fitter.GLSFitter(toas=ts1855, model=m1855)
glsfit.fit_toas(maxiter=1)
#glsfit.print_summary()

INFO: Applying clock corrections (include_GPS = True, include_BIPM = True) [pint.toa]
INFO: Observatory arecibo, loading clock file 
	/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pint/datafiles/time.dat [pint.observatory.topo_obs]
INFO: Applying observatory clock corrections. [pint.observatory.topo_obs]
INFO: Applying GPS to UTC clock correction (~few nanoseconds) [pint.observatory.topo_obs]
INFO: Observatory arecibo, loading GPS clock file 
	/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pint/datafiles/gps2utc.clk [pint.observatory.topo_obs]
INFO: Applying TT(TAI) to TT(BIPM) clock correction (~27 us) [pint.observatory.topo_obs]
INFO: Observatory arecibo, loading BIPM clock file 
	/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pint/datafiles/tai2tt_bipm2015.clk [pint.observatory.topo_obs]
INFO: Computing TDB columns. [pint.toa]
INFO: Doing astropy mode TDB conversion [pint.observatory]
INFO:

INFO: Set solar system ephemeris to link:
	ftp://ssd.jpl.nasa.gov/pub/eph/planets/bsp/de421.bsp [pint.solar_system_ephemerides]
INFO: Pickling TOAs. [pint.toa]


15146.9885449523521945

In [3]:
ts1855.print_summary()

Number of TOAs:  702
Number of commands:  1
Number of observatories:  1 ['arecibo']
MJD span:  53358.727 to 55108.922
Date span: 2004-12-19 17:27:32.966478287 to 2009-10-04 22:07:33.664872525
arecibo TOAs (702):
  Min freq:      420.000 MHz 
  Max freq:      1442.000 MHz 
  Min error:     0.03 us
  Max error:     82 us
  Median error:  1.2 us



In [4]:
def dmx_trial_receiver_and_frequency(times, verbose = True):    
    MJDs = times.get_mjds() 
    freqs = times.table["freq"].quantity
    divide_freq = ((0.5*(np.max(freqs) - np.min(freqs)))) 
    DMXs = []
    receiver = np.array(times.get_flag_value("be"))
    binwidth = []
    #different receivers require different binwidths
    for r in receiver:
        if r in ['ASP','AO','PUPPI']:
            #print(r)
            binwidth.append(0.5) #unit = days
        if r =='GASP':
            binwidth.append(15.0) #unit = days
        if r =='GUPPI':
            binwidth.append(6.5) #unit = days
       # if r in ['ASP','AO','PUPPI']:
        #    binwidth.append(0.5) #unit = days

    binwidth = np.array(binwidth) * u.d
    prevbinR2 = MJDs[0] - 0.001 * u.d
    
    while True:
        if not np.any(MJDs > prevbinR2):
            break
        startMJD = MJDs[MJDs > prevbinR2][0]
            #takes the first MJD after the end of the previous bin
        binidx = np.logical_and(MJDs > prevbinR2, MJDs <= startMJD + binwidth)
            #if MJDs > previous bin's MJD and MJD < start, it is its index
        if not np.any(binidx):
            break
        binMJDs = MJDs[binidx]
        binfreqs = freqs[binidx]
        loMJDs = binMJDs[binfreqs < divide_freq]
        hiMJDs = binMJDs[binfreqs >= divide_freq]
            # If we have freqs below and above the divide, this is a good bin
        if np.any(binfreqs < divide_freq) and np.any(binfreqs > divide_freq):
            DMXs.append(dmxrange(list(loMJDs), list(hiMJDs)))
        else:
                # These TOAs cannot be used
            pass
        prevbinR2 = binMJDs.max()
        
        if verbose:
            print(
                "\n These are the good DMX ranges with number of TOAs above/below the dividing freq:"
            )
            for DMX in DMXs:
                DMX.sum_print()

#####********BEGINNING OF [get rid of TOAs if f_max/f_min < 1.1 (inadequate bandwidth)]**********#
         
        fitter = glsfit
        model = fitter.model
        mjds = fitter.toas.get_mjds()
        freqs = fitter.toas.table["freq"]
        ii = 1

        while hasattr(model, "DMX_{:04d}".format(ii)):
            mmask = np.logical_and(
                mjds.value > getattr(model, "DMXR1_{:04d}".format(ii)).value,
                mjds.value < getattr(model, "DMXR2_{:04d}".format(ii)).value,
            )
            if np.any(mmask):
                mjds_in_bin = mjds[mmask]
                freqs_in_bin = freqs[mmask]
                freqratio = freqs_in_bin.max() / freqs_in_bin.min()
                if freqratio < 1.1:
                # print("DMX_{:04d}".format(ii, mmask.sum(), freqratio))
                    model.remove_param("DMX_{:04d}".format(ii,mmask.sum()))                
                ii += 1
#####********END OF [get rid of TOAs if f_max/f_min < 1.1 (inadequate bandwidth)]**********#
 
        #make the mask
        # Init mask to all False
        mask = np.zeros_like(MJDs.value, dtype=np.bool)
        # Mark TOAs as True if they are in any DMX bin
        for DMX in DMXs:
            mask[np.logical_and(MJDs >= DMX.min, MJDs <= DMX.max)] = True
        log.info("{} out of {} TOAs are in a DMX bin".format(mask.sum(), len(mask)))
        #Instantiate a DMX component
        dmx_class = Component.component_types["DispersionDMX"]
        dmx_comp = dmx_class()
        # Add parameters
        for ii, DMX in enumerate(DMXs):
            if ii == 0:
                # Already have DMX_0001 in component, so just set parameters
                dmx_comp.DMX_0001.value = 0.0
                dmx_comp.DMX_0001.frozen = False
                dmx_comp.DMXR1_0001.value = DMX.min.value
                dmx_comp.DMXR2_0001.value = DMX.max.value

            else:
                # Add the DMX parameters
                dmx_par = pint.models.parameter.prefixParameter(
                    parameter_type="float",
                    name="DMX_{:04d}".format(ii + 1),
                    value=0.0,
                    units=u.pc / u.cm ** 3,
                    frozen=False,
                )
                dmx_comp.add_param(dmx_par, setup=True)

                dmxr1_par = pint.models.parameter.prefixParameter(
                    parameter_type="mjd",
                    name="DMXR1_{:04d}".format(ii + 1),
                    value=DMX.min.value,
                    units=u.d,
                )
                dmx_comp.add_param(dmxr1_par, setup=True)

                dmxr2_par = pint.models.parameter.prefixParameter(
                    parameter_type="mjd",
                    name="DMXR2_{:04d}".format(ii + 1),
                    value=DMX.max.value,
                    units=u.d,
                )
                dmx_comp.add_param(dmxr2_par, setup=True)
            # Validate component
            dmx_comp.validate()

            return mask, dmx_comp

In [5]:
dmx_trial_receiver_and_frequency(ts1855, verbose = True)


 These are the good DMX ranges with number of TOAs above/below the dividing freq:
INFO: 0 out of 702 TOAs are in a DMX bin [unknown]

 These are the good DMX ranges with number of TOAs above/below the dividing freq:
53420.55-53420.58 (    0.03 d): NLO=    6 NHI=   15
INFO: 21 out of 702 TOAs are in a DMX bin [unknown]


(array([False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
      